# Climate Overview for Energy Modelling

Initial step to perform energy modelling is to get an overview of the climate data available for the region of interest. This notebook provides a guide to download and process ERA5-Land monthly data using the CDS API.


## 0. Getting ERA5-Land monthly via Python API

> You need to install the `cdsapi` package to use this notebook.

You can install it using `pip install cdsapi`, but you also need to register on the [Climate Data Store](https://cds.climate.copernicus.eu/api-how-to) and get your API key.
You need to do that only once, and it is valid for all future downloads.

The key should be saved in the `~/.cdsapirc` file in your home directory, but everything is explained in the link above.


In [38]:
import cdsapi
import timeit
import os
from utils_climatic import *
import xarray as xr
import zipfile
from matplotlib import pyplot as plt
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import calendar

### 1. Fill in the parameters

You only need to fill 3 information below to download the data and extract you need.

1. Define the region of interest using ISO A2 codes within the `ISO_A2` list.
2. Specify the start and end years for the data collection.
3. List the variables you want to download from the ERA5-Land dataset in the `variables_list`.

In [39]:
ISO_A2 = ['AO', 'BI', 'CM', 'CF', 'TD', 'CG', 'CD', 'GQ', 'GA', 'ST']
ISO_A2 = ['AO', 'BW', 'CD', 'SZ', 'LS', 'MW', 'MZ', 'NA', 'ZA', 'TZ', 'ZM', 'ZW']
start_year, end_year = 1950, 2025                                           # from 1950

variables_list = ['2m_temperature',
                  'total_precipitation'
                 ]# to present year

#### Processing parameters
- Prepare the folder structure
- Data will be downloaded in the `climatic/era5_api` folder
- Extracted data will be saved in the `climatic/era5_extract` folder
- Output data will be saved in the `climatic/output` folder
- Data will be downloaded from the ERA5-Land monthly means dataset

In [40]:

# Create folder structure
folder_input = 'input'
if not os.path.exists(folder_input): os.mkdir(folder_input)
folder_api = os.path.join(folder_input, 'era5_api')
if not os.path.exists(folder_api): os.mkdir(folder_api)
folder_extract = os.path.join(folder_input, 'era5_extract')
if not os.path.exists(folder_extract): os.mkdir(folder_extract)
folder_output = 'output'
if not os.path.exists(folder_output): os.mkdir(folder_output)
folder_output = os.path.join(folder_output, '_'.join(ISO_A2))
if not os.path.exists(folder_output): os.mkdir(folder_output)

# Define the dataset
dataset_name = 'reanalysis-era5-land-monthly-means'# 'reanalysis-era5-land'

# Define the API client variable
variable_name = {
    'total_precipitation': 'tp',
    'surface_runoff': 'sro',
    'runoff': 'ro',
    'snow_depth_water_equivalent': 'sd',
    '2m_temperature': 't2m',
    'potential_evaporation': 'pev',
    'total_evaporation': 'e'
}

#
temp = '_'.join([variable_name[variable] for variable in variables_list])
downloaded_files = {iso: f'{dataset_name}_{iso}_{start_year}_{end_year}_{temp}.zip' for iso in ISO_A2}
downloaded_files = {iso: os.path.join(folder_api, file) for iso, file in downloaded_files.items()}
print(f'Files will be download in: {folder_api}, files: {downloaded_files}')

# Define the bounding box for each ISO A2 code
locations = {iso: get_bbox(iso) for iso in ISO_A2}
print(f'Locations: {locations}')

Files will be download in: input/era5_api, files: {'AO': 'input/era5_api/reanalysis-era5-land-monthly-means_AO_1950_2025_t2m_tp.zip', 'BW': 'input/era5_api/reanalysis-era5-land-monthly-means_BW_1950_2025_t2m_tp.zip', 'CD': 'input/era5_api/reanalysis-era5-land-monthly-means_CD_1950_2025_t2m_tp.zip', 'SZ': 'input/era5_api/reanalysis-era5-land-monthly-means_SZ_1950_2025_t2m_tp.zip', 'LS': 'input/era5_api/reanalysis-era5-land-monthly-means_LS_1950_2025_t2m_tp.zip', 'MW': 'input/era5_api/reanalysis-era5-land-monthly-means_MW_1950_2025_t2m_tp.zip', 'MZ': 'input/era5_api/reanalysis-era5-land-monthly-means_MZ_1950_2025_t2m_tp.zip', 'NA': 'input/era5_api/reanalysis-era5-land-monthly-means_NA_1950_2025_t2m_tp.zip', 'ZA': 'input/era5_api/reanalysis-era5-land-monthly-means_ZA_1950_2025_t2m_tp.zip', 'TZ': 'input/era5_api/reanalysis-era5-land-monthly-means_TZ_1950_2025_t2m_tp.zip', 'ZM': 'input/era5_api/reanalysis-era5-land-monthly-means_ZM_1950_2025_t2m_tp.zip', 'ZW': 'input/era5_api/reanalysis-era

### 2. Automatic download from the API Climate Data Store

By default, the data will be downloaded in the `climatic/era5_api` folder. If you want to change the folder, please change the `folder_api` variable above.
It will be downloaded as a zip file including `.grib`files. Includes all monthly means for the specified period.

In [41]:
years = [ str(start_year +i ) for i in range(end_year - start_year + 1)]

# Create folder if not exists
if not os.path.exists(folder_api): os.mkdir(folder_api)

for (iso, (long_west, lat_south, long_east, lat_north)) in locations.items():

    downloaded_file = downloaded_files[iso]

    print(f'Processing {iso} with bbox: {long_west}, {lat_south}, {long_east}, {lat_north}')
    if not os.path.exists(downloaded_file):
        print('Process started. Please wait the ending message ... ')
        start = timeit.default_timer()
        c = cdsapi.Client()

        c.retrieve(
            dataset_name,
            {
                #'format': 'netcdf',
                'format': 'grib',
                'product_type': 'monthly_averaged_reanalysis',
                'variable': variables_list,
                'year': years,
                'month': [ '01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12' ],
                'time': '00:00',
                'area': [ lat_south, long_west, lat_north, long_east ],
            }, downloaded_file
            )

        stop = timeit.default_timer()
        print('Process completed in ', (stop - start)/60, ' minutes')
    else:
        print('File already exists.')

2025-06-26 17:48:41,733 WARNING [2025-06-23T00:00:00] Scheduled System Session affecting Service reliability - 30 June 2025. Please follow status [here](https://status.ecmwf.int/) or in our [forum](https://forum.ecmwf.int/t/scheduled-maintenance-of-the-cloud-infrastructure-on-30-june-2025/13598)
2025-06-26 17:48:41,734 INFO [2025-06-16T00:00:00] CC-BY licence to replace Licence to use Copernicus Products on 02 July 2025. More information available [here](https://forum.ecmwf.int/t/cc-by-licence-to-replace-licence-to-use-copernicus-products-on-02-july-2025/13464)
2025-06-26 17:48:41,734 INFO [2025-06-10T00:00:00] To improve our C3S service, we need to hear from you! Please complete this very short [survey](https://confluence.ecmwf.int/x/E7uBEQ/). Thank you.
2025-06-26 17:48:41,735 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.


Processing AO with bbox: 11.66939414300009, -18.031404723999884, 24.061714315000103, -4.391203714999932
File already exists.
Processing BW with bbox: 19.978345988000115, -26.891794127999987, 29.350073689000055, -17.781807555999947
File already exists.
Processing CD with bbox: 12.210541212000066, -13.45835052399994, 31.280446818000087, 5.375280253000085
File already exists.
Processing SZ with bbox: 30.782906128000093, -27.31626434299993, 32.11739831600005, -25.735999043999954
File already exists.
Processing LS with bbox: 27.002154989000132, -30.658799335999916, 29.435908244000075, -28.570761413999975
File already exists.
Processing MW with bbox: 32.66330814700012, -17.135335387999902, 35.90429895000011, -9.381235046999876
File already exists.
Processing MZ with bbox: 30.21384525500008, -26.86027150399994, 40.847992384000065, -10.469008070999905
File already exists.
Processing NA with bbox: 11.717621290000068, -28.959368183999885, 25.259780721000112, -16.951057230999936
Process started. 

2025-06-26 17:48:41,992 INFO Request ID is 74a0ae16-97db-478d-a92a-21a9d5959d63
2025-06-26 17:48:42,112 INFO status has been updated to accepted
2025-06-26 17:55:01,888 INFO status has been updated to successful
2025-06-26 17:55:03,485 WARNING [2025-06-23T00:00:00] Scheduled System Session affecting Service reliability - 30 June 2025. Please follow status [here](https://status.ecmwf.int/) or in our [forum](https://forum.ecmwf.int/t/scheduled-maintenance-of-the-cloud-infrastructure-on-30-june-2025/13598)
2025-06-26 17:55:03,486 INFO [2025-06-16T00:00:00] CC-BY licence to replace Licence to use Copernicus Products on 02 July 2025. More information available [here](https://forum.ecmwf.int/t/cc-by-licence-to-replace-licence-to-use-copernicus-products-on-02-july-2025/13464)
2025-06-26 17:55:03,486 INFO [2025-06-10T00:00:00] To improve our C3S service, we need to hear from you! Please complete this very short [survey](https://confluence.ecmwf.int/x/E7uBEQ/). Thank you.
2025-06-26 17:55:03,48

Process completed in  6.362161024298985  minutes
Processing ZA with bbox: 16.469981316000087, -46.96575286299992, 37.97779381600009, -22.126451924999927
Process started. Please wait the ending message ... 


2025-06-26 17:55:03,897 INFO Request ID is 5db06901-81a4-427a-b29b-bae4d4ba0c7f
2025-06-26 17:55:03,958 INFO status has been updated to accepted
2025-06-26 17:55:25,305 INFO status has been updated to running
2025-06-26 18:01:23,263 INFO status has been updated to successful
2025-06-26 18:01:27,686 WARNING [2025-06-23T00:00:00] Scheduled System Session affecting Service reliability - 30 June 2025. Please follow status [here](https://status.ecmwf.int/) or in our [forum](https://forum.ecmwf.int/t/scheduled-maintenance-of-the-cloud-infrastructure-on-30-june-2025/13598)
2025-06-26 18:01:27,686 INFO [2025-06-16T00:00:00] CC-BY licence to replace Licence to use Copernicus Products on 02 July 2025. More information available [here](https://forum.ecmwf.int/t/cc-by-licence-to-replace-licence-to-use-copernicus-products-on-02-july-2025/13464)
2025-06-26 18:01:27,687 INFO [2025-06-10T00:00:00] To improve our C3S service, we need to hear from you! Please complete this very short [survey](https://co

...

Process completed in  8.352843281250292  minutes
Processing ZM with bbox: 21.97987756300006, -18.069231871999946, 33.67420251500005, -8.194124042999917
Process started. Please wait the ending message ... 


2025-06-26 18:09:49,622 INFO Request ID is c75df783-7837-4ad2-923a-b24f5da44891
2025-06-26 18:09:49,702 INFO status has been updated to accepted
2025-06-26 18:10:10,868 INFO status has been updated to running
2025-06-26 18:16:08,324 INFO status has been updated to successful
2025-06-26 18:16:09,598 WARNING [2025-06-23T00:00:00] Scheduled System Session affecting Service reliability - 30 June 2025. Please follow status [here](https://status.ecmwf.int/) or in our [forum](https://forum.ecmwf.int/t/scheduled-maintenance-of-the-cloud-infrastructure-on-30-june-2025/13598)
2025-06-26 18:16:09,599 INFO [2025-06-16T00:00:00] CC-BY licence to replace Licence to use Copernicus Products on 02 July 2025. More information available [here](https://forum.ecmwf.int/t/cc-by-licence-to-replace-licence-to-use-copernicus-products-on-02-july-2025/13464)
2025-06-26 18:16:09,599 INFO [2025-06-10T00:00:00] To improve our C3S service, we need to hear from you! Please complete this very short [survey](https://co

Process completed in  6.345986221534743  minutes
Processing ZW with bbox: 25.219369751000045, -22.39733978299992, 33.04276818900013, -15.614808044999876
Process started. Please wait the ending message ... 


2025-06-26 18:16:09,820 INFO Request ID is acec5420-d879-478c-b3b9-725381dd620b
2025-06-26 18:16:09,877 INFO status has been updated to accepted
2025-06-26 18:16:18,501 INFO status has been updated to running
2025-06-26 18:22:29,350 INFO status has been updated to successful
                                                                                          

Process completed in  6.346141703465643  minutes


### 3. Extracting and formatting the data

In [42]:
dataset = {}
for iso, downloaded_file in downloaded_files.items():
    print(f'Processing {iso} with file: {downloaded_file}')
    data = extract_data(downloaded_file, step_type=True, extract_to=folder_extract)

    # Calculate resolution and add units
    calculate_resolution_netcdf(data, lat_name='latitude', lon_name='longitude')

    for var in data.data_vars:
        print(f"{var}: {data[var].attrs['units']}")

    # Convert units if necessary
    data = convert_dataset_units(data)

    # Export transformed data to NetCDF for future analysis
    data.to_netcdf(os.path.join(folder_output, f"{ os.path.splitext(os.path.basename(downloaded_file))[0]}.nc"))

    dataset.update({iso: data})


Processing AO with file: input/era5_api/reanalysis-era5-land-monthly-means_AO_1950_2025_t2m_tp.zip
Opening GRIB file: input/era5_extract/reanalysis-era5-land-monthly-means_AO_1950_2025_t2m_tp.grib
Opening GRIB file: input/era5_extract/reanalysis-era5-land-monthly-means_AO_1950_2025_t2m_tp.grib
Spatial resolution: 0.10000813008130116° lon x -0.09999999999999964° lat
Approximate spatial resolution:
10.89 km (lon_name) x -11.10 km (lat_name) at -11.23° lat
Temporal resolution: 31 days
t2m: K
tp: m
Processing BW with file: input/era5_api/reanalysis-era5-land-monthly-means_BW_1950_2025_t2m_tp.zip
Opening GRIB file: input/era5_extract/reanalysis-era5-land-monthly-means_BW_1950_2025_t2m_tp.grib
Opening GRIB file: input/era5_extract/reanalysis-era5-land-monthly-means_BW_1950_2025_t2m_tp.grib
Spatial resolution: 0.10001075268817061° lon x -0.10000000000000142° lat
Approximate spatial resolution:
10.27 km (lon_name) x -11.10 km (lat_name) at -22.34° lat
Temporal resolution: 31 days
t2m: K
tp: m


### 4. Visualization of climate data

In [43]:
# Plotting spatial mean timeseries for all variables
for var in variables_list:
    # Find the indirect from the variable name
    plot_spatial_mean_timeseries_all_iso(dataset, var=variable_name.get(var, var), folder=folder_output)

    if var == '2m_temperature':
        agg = 'avg'
    elif var == 'total_precipitation':
        agg = 'sum'
    else:
        agg = None

    plot_spatial_mean_timeseries_all_iso(dataset, var=variable_name.get(var, var), agg=agg, folder=folder_output)

# Plotting scatter plot of annual spatial means
scatter_annual_spatial_means(dataset, var_x='t2m', var_y='tp', folder=folder_output)

# Plotting monthly average for each variable
plot_monthly_mean(dataset, "t2m", lat_name='latitude', lon_name='longitude', folder=folder_output)
plot_monthly_mean(dataset, "tp", lat_name='latitude', lon_name='longitude', folder=folder_output)

# Plotting monthly climatology for each variable
for iso, data in dataset.items():
    print(f'Processing {iso}')
    plot_monthly_climatology_grid(data, "t2m", filename=os.path.join(folder_output, f'monthly_climatology_{iso}_{var}.pdf'))
    plot_monthly_climatology_grid(data, "tp", filename=os.path.join(folder_output, f'monthly_climatology_{iso}_{var}.pdf'))


Processing AO
Processing BW
Processing CD
Processing SZ
Processing LS
Processing MW
Processing MZ
Processing NA
Processing ZA
Processing TZ
Processing ZM
Processing ZW


### 5. Checking representative regions

In [44]:
def plot_monthly_precipitation_heatmap(dataset: dict,
                                       var: str = 'tp',
                                       cmap: str = 'cividis',
                                       lat_name='latitude',
                                       lon_name='longitude',
                                       path=None):
    """
    Plot a heatmap of total monthly precipitation across countries.

    Parameters:
    -----------
    dataset : dict
        Dictionary where keys are country ISO codes or names, and values are xarray Datasets
        with a time dimension and a variable representing precipitation.

    var : str
        Name of the variable to extract (e.g., 'precip').

    cmap : str
        Colormap for the heatmap. Default is 'cividis' (colorblind-friendly).

    Returns:
    --------
    Displays a heatmap (countries x months) with total precipitation.
    """
    data_rows = []

    for iso, ds in dataset.items():
        if var not in ds:
            continue

        da = ds[var]
        spatial_mean = da.mean(dim=[lat_name, lon_name])
        # Group by calendar month and sum across time
        monthly_total = spatial_mean.groupby('time.month').mean(dim='time')

        # Convert to pandas Series (index = month, value = total precip)
        monthly_series = monthly_total.to_series()

        # Ensure all months are present (fill missing with 0)
        for month in range(1, 13):
            value = monthly_series.get(month, 0.0)
            data_rows.append({'country': iso, 'month': month, 'precip': value})

    # Create DataFrame and pivot
    df = pd.DataFrame(data_rows)
    heatmap_data = df.pivot(index='country', columns='month', values='precip').fillna(0)
    heatmap_data = heatmap_data.reindex(columns=range(1, 13))  # ensure 1–12 in order

    # Plot heatmap
    plt.figure(figsize=(12, max(4, 0.5 * len(heatmap_data))))
    sns.heatmap(heatmap_data, cmap=cmap, annot=True, fmt=".0f",
                cbar_kws={'label': 'Mean Precipitation (mm/day)'}, linewidths=0.5)

    plt.title("Monthly Mean Precipitation per Country (mm/day)")
    plt.xlabel("Month")
    plt.ylabel("Country")
    plt.xticks(
        ticks=[i + 0.5 for i in range(12)],
        labels=[calendar.month_abbr[i + 1] for i in range(12)],
        rotation=0
    )
    plt.yticks(rotation=0)
    plt.tight_layout()
    if path is not None:
        plt.savefig(path)
        plt.close()
    else:
        plt.show()


plot_monthly_precipitation_heatmap(dataset, path=os.path.join(folder_output, 'monthly_precipitation_heatmap.png'))